In [2]:
!pip install plotly==5.18.0

     ---------------------------------------- 15.6/15.6 MB 7.5 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.9.0
    Uninstalling plotly-5.9.0:
      Successfully uninstalled plotly-5.9.0


## Подготовка данных

In [126]:
import pandas as pd
import numpy as np
import plotly.express as px
import json
import ast

In [127]:
orders = pd.read_csv('orders.csv', parse_dates=[1])
sales = pd.read_csv('sales.csv', parse_dates=[1])
user = pd.read_csv('user.csv', parse_dates=[5])
store = pd.read_csv('store.csv', parse_dates=[4, 5])
product = pd.read_csv('product.csv')

In [128]:
def parse_json_field(df: pd.DataFrame, colunm_name: str) -> pd.DataFrame:
    # replacing trash
    df[colunm_name] = df[colunm_name].str.replace('\'', '\"')
    df[colunm_name] = df[colunm_name].str.replace('False', '0')
    df[colunm_name] = df[colunm_name].str.replace('True', '1')
    # eploding list of jsons
    df[colunm_name] = df[colunm_name].apply(ast.literal_eval)
    df = df.explode(colunm_name, ignore_index = True)
    # concat normalized dataframe
    return pd.concat([df.drop(colunm_name, axis=1), pd.json_normalize(df[colunm_name])], axis=1)

In [129]:
orders = parse_json_field(orders, 'product_info')

In [130]:
full_df = pd.merge(left=orders, right=user[['user_id', 'user_birthdate']], on='user_id')
full_df = pd.merge(left=full_df, right=product[['product_id', 'product_name']], on='product_id')
full_df = pd.merge(left=full_df, right=sales.drop(columns=['id']), on=['order_id', 'product_id'])
full_df = pd.merge(left=full_df, right=store, on='store_id')

full_df['full_price'] = full_df['product_price'] * full_df['count']
full_df['month'] = full_df['purchase_date'].dt.to_period('M')
full_df['full_revenue'] = np.where(full_df['comission_is_percent'] == True,
         (full_df['product_comission'] / 100 * full_df['product_price']) * full_df['count'],
         full_df['product_comission'] * full_df['count'])
full_df.head()

,order_id,order_date,user_id,store_id,product_id,product_price,product_comission,comission_is_percent,count,user_birthdate,...,is_canceled,is_accepted,longtitude,latitude,store_name,store_open_date,store_close_date,full_price,month,full_revenue
0,1,2022-01-11,13,6,3283,18245.907938,3000,0,1,1990-04-18,...,False,True,55.663782,37.511308,Rio Leninsky,2022-01-10,NaT,18245.907938,2022-01,3000.000000
1,234475,2023-04-29,1074,6,3283,18245.907938,75,1,1,1967-01-13,...,False,True,55.663782,37.511308,Rio Leninsky,2022-01-10,NaT,18245.907938,2023-05,13684.430953
2,128498,2023-01-16,4396,6,3283,18245.907938,4000,0,1,1984-01-03,...,True,True,55.663782,37.511308,Rio Leninsky,2022-01-10,NaT,18245.907938,2023-01,4000.000000
3,35555,2022-07-13,5334,6,3283,18245.907938,5,1,1,1972-11-20,...,False,True,55.663782,37.511308,Rio Leninsky,2022-01-10,NaT,18245.907938,2022-07,912.295397
4,95351,2022-11-24,41,6,3283,18245.907938,75,1,1,1971-09-27,...,False,True,55.663782,37.511308,Rio Leninsky,2022-01-10,NaT,18245.907938,2022-11,13684.430953


## Информация по пользователям

#### Средний возраст покупателей

In [138]:
print(f'Средний возраст покупателей = {(pd.to_datetime("today").normalize() - full_df.user_birthdate.mean()).days / 365} лет')

Средний возраст покупателей = 46.8 лет


### Сегментация по покупкам

Пользователей которые ни разу не совершали покупку не оказалось.

In [139]:
np.setdiff1d(full_df.query('is_accepted == False').user_id.unique(), full_df.query('is_accepted == True').user_id.unique())

array([], dtype=int64)

## Конверсия в покупку и процент выкупа среди магазинов

In [134]:
pd.concat([
    np.round((1 - full_df.query('is_accepted == True').groupby('store_name').agg(
        Конверсия_в_покупку=('order_id', 'count'),
    ) / full_df.groupby('store_name').agg(
        Конверсия_в_покупку=('order_id', 'count'),
    ) ), 4), 
    np.round((1 - full_df.query('is_canceled == True and is_accepted == True').groupby('store_name').agg(
        Процент_выкупа=('order_id', 'count'),
    ) / full_df.query('is_accepted == True').groupby('store_name').agg(
        Процент_выкупа=('order_id', 'count'),
    ) ), 4)], axis=1).style.format({
        'Конверсия_в_покупку': '{:,.2%}'.format,
        'Процент_выкупа': '{:,.2%}'.format,
    })

,Конверсия_в_покупку,Процент_выкупа
store_name,,
Afimoll,2.56%,22.51%
Aviapark,10.28%,68.60%
City Mall,59.07%,0.00%
Evropeisky,12.28%,63.68%
Evropolis,5.53%,67.88%
Galereia,15.67%,65.98%
Grinvich,26.84%,54.20%
Kapitoliy Vernadskogo,40.93%,40.18%
Kapitoly Belyaevo,5.13%,67.78%


## Тенденции показателей магазинов во времени
#### AOV

In [169]:
by_month = full_df.query(
    'is_canceled == False and is_accepted == True').groupby([
    'store_name', 'month'], as_index=False).agg(
    total_orders_price=('full_price', np.sum),
    total_orders_revenue=('full_revenue', np.sum),
    orders_count=('order_id', pd.Series.nunique),
)
by_month['AOV'] = np.round(by_month['total_orders_revenue'] / by_month['orders_count'], 2)

In [167]:
px.line(
    by_month,
    x=by_month['month'].dt.to_timestamp(),
    y='AOV',
    color='store_name',
    title='Store\'s AOV by months',
    labels={'x': 'month'},
    hover_data={
        'total_orders_price': ':,.2f',
        'total_orders_revenue': ':,.2f',
        'orders_count': ':,',
        'AOV': ':,.2f',
    }
)

### orders_count

In [168]:
px.line(
    by_month,
    x=by_month['month'].dt.to_timestamp(),
    y='orders_count',
    color='store_name',
    title='Store\'s orders count by months',
    labels={'x': 'month'},
    hover_data={
        'total_orders_price': ':,.2f',
        'total_orders_revenue': ':,.2f',
        'orders_count': ':,',
        'AOV': ':,',
    }
)

## Метрики магазинов

In [220]:
by_store = full_df.query(
    'is_canceled == False and is_accepted == True').groupby(
    'store_name', as_index=False).agg(
    total_price=('full_price', np.sum),
    total_revenue=('full_revenue', np.sum),
    products_count=('count', np.sum),
    orders_count=('order_id', pd.Series.nunique),
    longtitude=('longtitude', np.max),
    latitude=('latitude', np.max)
)

by_store['avg_price'] = np.round(by_store['total_price'] / by_store['orders_count'], 2)
by_store['avg_revenue'] = np.round(by_store['total_revenue'] / by_store['orders_count'], 2)
by_store['avg_products_count'] = np.round(by_store['products_count'] / by_store['orders_count'], 2)

In [219]:
fig = px.scatter_mapbox(
    by_store,
    lat="longtitude",
    lon="latitude",
    color="orders_count",
    opacity=.9,
    size="orders_count",
    zoom=3.5,
    mapbox_style="carto-positron",
    title='Stores',
    color_continuous_scale=px.colors.sequential.Emrld,
    hover_data={
        'store_name': True,
        'longtitude': False,
        'latitude': False,
        'total_price': ':,.2f',
        'total_revenue': ':,.2f',
        'total_revenue': ':,.2f',
        'products_count': ':,',
        'orders_count': ':,',
        'avg_price': ':,.2f',
        'avg_products_count': ':,.2f',
    }
)
fig.show()

## Cколько единиц каждого товара необходимо поставить в каждый из магазинов в следующем месяце?

Уберем из датасета декабрь 2023 года т.к. мы имеем данные не за весь месяц

In [240]:
top_products = full_df.query(
    'is_accepted == True').groupby([
    'store_name', 'month', 'product_name'], as_index=False).agg(
    product_count=('count', np.sum),
    product_margin=('full_revenue', np.sum),
)
top_products = top_products[top_products['month'] != "2023-12"].sort_values(
    by=['store_name', 'month', 'product_count'],
    ascending=[True, True, False]
)
top_products['month'] = top_products['month'].dt.month
top_products

,store_name,month,product_name,product_count,product_margin
3,Afimoll,10,Blossom Breeze Romper,256,1.364649e+06
19,Afimoll,10,Mars Mist Military Jacket,234,1.770430e+06
10,Afimoll,10,Dewdrop Dream Camisole,229,1.040765e+06
13,Afimoll,10,Emberstone Expedition Epaulettes,222,1.666579e+06
16,Afimoll,10,Gravity Guardian Gloves,212,9.025687e+05
...,...,...,...,...,...
9816,Tsvetnoy,11,Midnight Gala Dress,549,1.446459e+06
9817,Tsvetnoy,11,Midnight Momentum T-shirt,547,2.783963e+06
9821,Tsvetnoy,11,Nebula Navigator Jacket,544,1.016894e+06
9818,Tsvetnoy,11,Monsoon Mirage Chinos,540,2.282388e+06


Многие товары имеют сезонность, поэтому при выборе едениц товара для следующего месяца будем опираться на те же месяца прошлых лет

In [242]:
top_products.groupby([
    'store_name', 'month', 'product_name'], as_index=False).agg(
    product_count=('product_count', np.mean)
)

,store_name,month,product_name,product_count
0,Afimoll,1,Astral Ascent Anorak,364.0
1,Afimoll,1,Aurora Borealis Jacket,386.0
2,Afimoll,1,Azure Bliss Jeans,291.0
3,Afimoll,1,Blossom Breeze Romper,328.0
4,Afimoll,1,Celestial Silk Blouse,294.0
...,...,...,...,...
7063,Tsvetnoy,11,Velvet Vortex Blazer,555.0
7064,Tsvetnoy,11,Whispering Willow Palazzo Pants,603.0
7065,Tsvetnoy,11,Wilderness Voyager Waistcoat,566.0
7066,Tsvetnoy,11,Wilderness Voyager Windbreaker,612.0


### Топ 10 самых популярных товаров

In [246]:
top_products.groupby(['product_name'], as_index=False).product_count.sum().sort_values(
    by='product_count',
    ascending=False
)[:10]

,product_name,product_count
8,Crimson Cascade Crewneck,48072
56,Velvet Elegance Trousers,48002
45,Slate Solace Sweater,47983
2,Azure Bliss Jeans,47895
42,Savvy Summit Ski Jacket,47854
38,Radiant Raven Rugby Polo,47838
4,Celestial Silk Blouse,47790
39,Rogue Rendezvous Raincoat,47722
32,Onyx Overdrive Overcoat,47668
27,Nomad Nebula Henley,47656
